In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import pickle
sys.path.append('../../')
print(sys.path)
import IPython.display as ipd
import numpy as np
from mlutils.utils import plot, plots
from models.mlmodeldic import best_model_path_and_settings
import torch
from tts.Alternative.neuravoice import Vocoder
from tts.Alternative.train import train_vocoder
from tts.Alternative.dataset import NeuraVoiceDataset
from torch.utils.data import DataLoader
import time

['/var/Sources/experiments/core/tts/Alternative', '/home/kaz/anaconda3/envs/core/lib/python37.zip', '/home/kaz/anaconda3/envs/core/lib/python3.7', '/home/kaz/anaconda3/envs/core/lib/python3.7/lib-dynload', '', '/home/kaz/.local/lib/python3.7/site-packages', '/home/kaz/anaconda3/envs/core/lib/python3.7/site-packages', '/home/kaz/anaconda3/envs/core/lib/python3.7/site-packages/Mako-1.0.7-py3.7.egg', '/home/kaz/anaconda3/envs/core/lib/python3.7/site-packages/IPython/extensions', '/home/kaz/.ipython', '../../']


In [2]:
batch_size = 16

In [3]:
cuda = 3

In [4]:
hidden_size = 512

In [5]:
lr = 1e-4

In [6]:
model_name = f'vocoder_{hidden_size}_{batch_size}_{lr}'

In [7]:
dataset = NeuraVoiceDataset(batch_size=batch_size)
loader = DataLoader(
    dataset, batch_size=batch_size, shuffle=True, collate_fn=dataset.mel_to_wav, drop_last=False, num_workers=3)

In [8]:
model_path, settings = best_model_path_and_settings(model_name, 'loss_ave', is_lower_better=True)
model_path, settings

(None, None)

In [ ]:
device = f'cuda:{cuda}'

In [ ]:
use_best_model = False

In [ ]:
if model_path and use_best_model: model = Vocoder.init_from_settings(settings, model_path, device=device)
else:                             model = Vocoder(hidden_size=hidden_size, device=device)

Trainable Parameters: 3.744 million


In [ ]:
losses, loss_aves, model = train_vocoder(
    model, loader, lr=lr, model_name=model_name, n_epoch=100, seqlen=2500, device=device, verbose=False, check_inference=True)

epoch 0/99 iter: 0/7614 total_iter: 100-- loss ave: 5.7100 loss: 5.30 -- elapse: 0m 44s speed 2.3 steps/sec
epoch 0/99 iter: 0/7614 total_iter: 200-- loss ave: 4.5268 loss: 2.32 -- elapse: 1m 13s speed 2.7 steps/sec
epoch 0/99 iter: 0/7614 total_iter: 300-- loss ave: 3.5327 loss: 1.21 -- elapse: 1m 41s speed 2.9 steps/sec
epoch 0/99 iter: 0/7614 total_iter: 400-- loss ave: 2.8971 loss: 0.60 -- elapse: 2m 11s speed 3.1 steps/sec
samples [[104 104 104 ...  85  85  85]
 [193 196 196 ... 107 170 107]
 [ 71  71 193 ...  57  53  51]
 ...
 [107  57  71 ... 170 170 170]
 [ 51  51  53 ... 107 107 107]
 [ 79  79 170 ... 193 193 193]]
y [[ 95.  90.  85. ...  82.  80.  78.]
 [204. 203. 201. ... 157. 115. 121.]
 [ 76. 187. 186. ...  51.  46.  42.]
 ...
 [ 63.  78. 162. ... 153. 153. 154.]
 [ 52.  59.  69. ... 116. 110. 106.]
 [ 79. 174. 166. ... 191. 194. 198.]]
loss 5.089479446411133
epoch 0/99 iter: 16/7614 total_iter: 500-- loss ave: 5.1374 loss: 5.09 -- elapse: 2m 39s speed 3.1 steps/sec
epoch 

In [ ]:
plots([losses, loss_aves], labels=['losses', 'loss_aves'])

#### 